## CP_APR, CP_ALS, HOSVD, & TUCKER_ALS Profiling Visualization with gprof2dot

In [9]:
import cProfile
import glob
import os
import pstats
from typing import Callable, Dict, List, Optional, Union

from pyttb import cp_als, cp_apr, hosvd, import_data, sptensor, tensor, tucker_als

In [10]:
def get_algorithm_func(algorithm_name: str) -> Optional[Callable]:
    """
    Returns the corresponding function for the user-supplied algorithm name.

    Parameters
    ----------
    algorithm_name:
        The algorithm to profile: 'cp_apr', 'cp_als', 'tucker_als', or 'hosvd'.

    Returns
    -------
    alg_func:
        The function corresponding to the algorithm.
    """

    # input validation
    func_handler = {
        "cp_apr": cp_apr,
        "cp_als": cp_als,
        "tucker_als": tucker_als,
        "hosvd": hosvd,
    }

    alg_func = func_handler.get(algorithm_name.lower())
    if alg_func is None:
        raise ValueError(f"'{algorithm_name}' is not a recognized algorithm.")
    return alg_func

In [11]:
def profile_alg(
    alg_func: Callable,
    input_tensor: Union[sptensor, tensor],
    test_file: str,
    algorithm_name: str,
    **params: Optional[Dict[str, Union[int, float]]],
) -> None:
    """
    Profiles the performance of the specified algorithm and prints the statistics.

    Parameters
    ----------
    alg_func:
        The function to profile.
    input_tensor:
        The input data tensor provided to the alg_func.
    test_file:
        The name of the tensor file.
    algorithm_name:
        The name of the user-supplied algorithm.
    params:
        Paramters passed to the algorithm function.
        'rank' may be given to the CP algorithms; 'tol' and 'verbosity' to hosvd.
    """

    # initialize a cProfile object and start collecting profiling data.
    profiler = cProfile.Profile()
    profiler.enable()

    alg_func(input_tensor, **params)

    # stop collecting data, and send data to Stats object and sort
    profiler.disable()

    # save profiling ouput to sub-directory.
    output_directory = "./profiling_output_files"
    if not os.path.exists(output_directory):
        os.makedirs(output_directory)  # create directory if it doesn't exist

    # from 'foo_tensor_10_4.tns' obtain 'foo_tensor_10_4'
    tf_basename_without_tns_ext = os.path.basename(test_file).split(".")[0]
    output_file = (
        f"{output_directory}/{tf_basename_without_tns_ext}_{algorithm_name}.pstats"
    )

    # write profiling results to new file in output_directory.
    profiler.dump_stats(output_file)

    print(
        f"Profiling stats for '{algorithm_name}' on '{os.path.basename(test_file)}' saved to '{output_file}'"
    )

In [12]:
def profile(
    test_files: List[str],
    ranks: List[int],
    algorithm_name: str,
    **params: Optional[Dict[str, Union[int, float]]],
):
    """
    Profiles the performance of the cp and Tucker algorithms with a set of tensors from test_files and ranks.

    Parameters
    ----------
    test_files:
        A list of strings representing the file paths to the test tensors.
    ranks:
        A list of integers representing the tensor testing ranks.
    algorithm_name:
        The algorithm to profile. Should be either 'cp_apr' or 'cp_als'.
    params:
        Paramters passed to the algorithm function.
        'rank' may be given to the CP algorithms; 'tol' and 'verbosity' to hosvd.
    """

    # obtain the appropriate function.
    alg_func = get_algorithm_func(algorithm_name)

    # choose only 'integer' files for cp_apr.
    if algorithm_name == "cp_apr":
        test_files = [tf for tf in test_files if "integer" in tf]
    # TODO: bypassing a "TypeError: unsupported operand type(s) for ** or pow(): 'sptensor' and 'int'."
    if algorithm_name == "hosvd":
        test_files = [tf for tf in test_files if "sparse" not in tf]

    for test_file in test_files:
        print("*" * 80)
        try:
            input_tensor = import_data(test_file)  # Load the tensor.
            if algorithm_name != "hosvd":
                # test across ranks for non-hosvd algos, since hosvd doesn't accept 'rank'.
                for rank in ranks:
                    # load the rank parameter to the testing algorithm's params
                    params["rank"] = rank
                    profile_alg(
                        alg_func, input_tensor, test_file, algorithm_name, **params
                    )
            else:
                profile_alg(alg_func, input_tensor, test_file, algorithm_name, **params)
        except Exception as e:
            print(
                f"Error when testing {os.path.basename(test_file)} for Algorithm = {algorithm_name}: {type(e).__name__}: {e}"
            )

### Read in Test Data

In [13]:
ranks = [2, 3, 4]
test_files = glob.glob("data/*.tns")

### Testing with Default Parameters

In [ ]:
profile(test_files, ranks, "cp_apr")

In [ ]:
profile(test_files, ranks, "cp_als")

In [14]:
profile(test_files, ranks, "tucker_als")

********************************************************************************

Tucker Alternating Least-Squares:

 Iter 0: fit = 5.787765e-02 fitdelta = 5.8e-02

 Iter 1: fit = 1.043565e-01 fitdelta = 4.6e-02

 Iter 2: fit = 1.079319e-01 fitdelta = 3.6e-03

 Iter 3: fit = 1.086132e-01 fitdelta = 6.8e-04

 Iter 4: fit = 1.087804e-01 fitdelta = 1.7e-04

 Iter 5: fit = 1.088221e-01 fitdelta = 4.2e-05

Profiling stats for 'tucker_als' on 'sptensor_sparse_continuous_size_10_8_6_4_rng_1.tns' saved to './profiling_output_files/sptensor_sparse_continuous_size_10_8_6_4_rng_1_tucker_als.pstats'

Tucker Alternating Least-Squares:

 Iter 0: fit = 1.412753e-01 fitdelta = 1.4e-01

 Iter 1: fit = 2.301989e-01 fitdelta = 8.9e-02

 Iter 2: fit = 2.402491e-01 fitdelta = 1.0e-02

 Iter 3: fit = 2.404686e-01 fitdelta = 2.2e-04

 Iter 4: fit = 2.404865e-01 fitdelta = 1.8e-05

Profiling stats for 'tucker_als' on 'sptensor_sparse_continuous_size_10_8_6_4_rng_1.tns' saved to './profiling_output_files/spten

In [ ]:
profile(test_files, ranks, "hosvd", tol=1e-4)

### **CP_APR**: Decreased stoptol, increased maxiters and maxinneriters, and differing input algorithms.

`cp_apr`'s default parameters are:
- **algorithm** = "mu"
- **stoptol** = 1e-4
- **maxiters** = 1000
- **maxinneriters** = 10

In [ ]:
profile(
    test_files,
    ranks,
    "cp_apr",
    algorithm="mu",
    stoptol=1e-5,
    maxiters=1200,
    maxinneriters=12,
)

In [ ]:
profile(
    test_files,
    ranks,
    "cp_apr",
    algorithm="pdnr",
    stoptol=1e-5,
    maxiters=2000,
    maxinneriters=20,
)

In [ ]:
profile(
    test_files,
    ranks,
    "cp_apr",
    algorithm="pqnr",
    stoptol=1e-5,
    maxiters=2000,
    maxinneriters=20,
)

### **CP_ALS**: Decreased stoptol, increased maxiters.
`cp_als`'s default parameters are:
- **stoptol** = 1e-4
- **maxiters** = 1000

In [ ]:
profile(test_files, ranks, "cp_als", stoptol=1e-5, maxiters=2000)

### **TUCKER_ALS**: Decreased stoptol, increased maxiters.

`tucker_als`'s default parameters are:
- **stoptol** = 1e-4
- **maxiters** = 1000

In [ ]:
profile(test_files, ranks, "tucker_als", stoptol=1e-5, maxiters=2000)

### **HOSVD**: Decreasing tol.

In [ ]:
profile(test_files, ranks, "hosvd", tol=1e-6)

## Visualising the profiling output with ***gprof2dot*** 
`/profiling_ouput_files/*.pstats` images generated to `/output_images` by running this in Terminal from within `/profiling` directory:
```bash
mkdir -p profiling_images && for file in profiling_output_files/*.pstats; do gprof2dot -f pstats $file \
    | dot -Tpng -o profiling_images/$(basename $file .pstats).png; done
```
**Based on [***gprof2dot*** instructions](https://nesi.github.io/perf-training/python-scatter/profiling-cprofile).**